In [27]:
import random
import logging
import sys
import numpy as np
from typing import Callable
import heapq
from collections import Counter
logging.basicConfig(format="%(message)s", level=logging.INFO)

In [28]:
N=50
MIN_NUMBER = sys.float_info.min
#print(MIN_NUMBER)

In [29]:
# Copyright © 2022 Giovanni Squillero <squillero@polito.it>
# https://github.com/squillero/computational-intelligence
# Free for personal or classroom use; see 'LICENSE.md' for details.


class PriorityQueue:
    """A basic Priority Queue with simple performance optimizations"""

    def __init__(self):
        self._data_heap = list()
        self._data_set = set()

    def __bool__(self):
        return bool(self._data_set)

    def __contains__(self, item):
        return item in self._data_set

    def push(self, item, p=None):
        assert item not in self, f"Duplicated element"
        if p is None:
            p = len(self._data_set)
        self._data_set.add(item)
        heapq.heappush(self._data_heap, (p, item))

    def pop(self):
        p, item = heapq.heappop(self._data_heap)
        self._data_set.remove(item)
        return item

In [30]:
def problem(N, seed=42):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [31]:
GOAL={i for i in range(N)}
print(GOAL)
list_of_lists=problem(N)
list_of_lists=list(set(map(tuple,list_of_lists)))
print(list_of_lists)
N_of_lists=len(list_of_lists)
print(len(list_of_lists))


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49}
[(34, 35, 3, 37, 9, 42, 44, 45, 46, 12, 16, 13, 18, 47, 20, 27, 29, 31), (32, 38, 42, 13, 48, 17, 26, 27, 28, 31), (2, 6, 13, 14, 16, 19, 20, 22, 23, 26, 29, 30, 31, 32, 34, 36, 41, 46, 48), (4, 5, 9, 13, 15, 23, 26, 28, 29, 30, 33, 34, 35, 38, 42, 43, 44, 45, 47, 48), (1, 5, 11, 13, 15, 17, 20, 21, 22, 24, 27, 28, 29, 31, 36, 37, 39, 42, 46, 48), (33, 34, 3, 39, 42, 12, 48, 23, 26, 29), (1, 3, 38, 7, 15, 19, 20, 23, 31), (34, 2, 4, 5, 6, 39, 40, 43, 45, 14, 15, 48, 47, 18, 27, 31), (3, 4, 38, 41, 10, 43, 44, 13, 15, 19, 21, 25, 26, 28, 29, 30), (32, 1, 3, 5, 37, 38, 41, 43, 13, 15, 18, 22, 25, 29, 30, 31), (35, 39, 12, 45, 14, 15, 13, 17, 49, 19, 18, 44, 20, 30, 31), (32, 34, 35, 37, 5, 8, 14, 47, 15, 16, 48, 23, 25, 27, 31), (32, 38, 37, 5, 39, 8, 7, 9, 43, 11, 45, 44, 15, 19, 20, 23, 24, 26), (0

In [32]:
def concat_list_of_lists(state):
    c_list = []
    for n in state:
        c_list += list_of_lists[n]
    return c_list

def print_lists(state):
    for n in state:
        print("List number: ",n, " with elements ", list_of_lists[n])
        

In [33]:
def goal_test(state):
    if set(concat_list_of_lists(state)) == GOAL:
        return True
    return False

In [34]:
def result(state):
    new_states=[]
    for n in range(state[len(state)-1]+1, len(list_of_lists)):
        N=(n,)
        new_state=state+N
        new_states.append(new_state)
    return new_states

#state=(0,)
#print(result(state))

In [35]:
def calculate_cost(state):
    c_list=concat_list_of_lists(state)
    repetitions=len(c_list)-len(set(c_list))
    if repetitions==0:
        repetitions=MIN_NUMBER
    normalized_cost=repetitions/len(c_list)
    return normalized_cost
    

In [36]:
def search(
    goal_test: Callable,
    state_cost: dict,
    priority_function: Callable,
    calculate_cost: Callable,
):  
    #initialize collections
    frontier = PriorityQueue()
    state_cost.clear()
    #keeping track of the iterations/nodes
    n=1
    N_nodes=len(list_of_lists)
    #INITIAL_STATE
    for s in range(0,len(list_of_lists)):
        S=(s,)
        state_cost[hash(S)] = calculate_cost(S)
        #print(S)
        #print(state_cost[hash(S)])
        frontier.push(S,p=priority_function(S))
    state=frontier.pop()
    #print(state)
    #print(state_cost[hash(state)])
        
    #BEGIN OF THE LOOP
    while state is not None and not goal_test(state):
        n+=1
        new_states = result(state)
        for new_state in new_states:
            if hash(new_state) not in state_cost and new_state not in frontier:
                state_cost[hash(new_state)] = calculate_cost(new_state) 
                frontier.push(new_state, p=priority_function(new_state))
                N_nodes+=1
                #logging.debug(f"Added new node to frontier (cost={state_cost[hash(new_state)]})")
        if frontier:
            state = frontier.pop()
        else:
            state = None
    
    #OUTPUT       
    if state is not None:
        c_list=concat_list_of_lists(state)
        repetitions=len(c_list)-len(set(c_list))
        logging.info(f"Found a solution for N={N:,} with {n:,} processed states, with {N_nodes:,} nodes added in the frontier and with {repetitions:,} repetitions.")
        print("The lists are:")
        print_lists(state)
    else:
        logging.info(f"No solution found")
    

In [37]:
state_cost = dict()

search(
    goal_test=goal_test,
    state_cost=state_cost,
    priority_function=lambda s: state_cost[hash(s)],
    calculate_cost=calculate_cost,
)


Found a solution for N=50 with 1,033,070 processed states, with 49,392,946 nodes added in the frontier and with 15 repetitions.


The lists are:
List number:  92  with elements  (1, 2, 40, 47, 16, 22, 23, 24, 30)
List number:  117  with elements  (32, 34, 3, 36, 38, 6, 8, 41, 10, 43, 45, 48, 17, 18, 20, 26)
List number:  134  with elements  (35, 4, 37, 42, 12, 13, 46, 48, 21, 30)
List number:  174  with elements  (2, 3, 4, 5, 38, 39, 41, 18, 19, 23, 28, 29)
List number:  208  with elements  (0, 33, 3, 35, 6, 7, 41, 9, 11, 44, 14, 15, 48, 49, 25, 27, 29, 31)
